In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load training and test data
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')

In [3]:
# response variable
y = train.solved_status

## Prepare a very simple model based on just the accuracy of the problem

In [4]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [5]:
model = build_extreme_gradient_boosting(train, test)

In [6]:
scores = eval_models([model], train, y)

accuracy score: 0.860933
combined score: 0.860933
accuracy score: 0.863600
combined score: 0.863600
accuracy score: 0.857933
combined score: 0.857933
accuracy score: 0.858000
combined score: 0.858000
accuracy score: 0.857467
combined score: 0.857467


In [7]:
# train model
import time

start_time = time.time()
model.fit(train, y)
elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

It took 4.346000 seconds to train the model 


In [8]:
predictions  = model.predict(test)

In [9]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': predictions.astype(int)})
submission_df.to_csv('submissions/fourth_xgb.csv', index=False)

In [ ]:
user_problem_train.level.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
lbl.fit(user_problem_train.level)

levels = lbl.transform(user_problem_train.level)

In [ ]:
pd.Series(levels).value_counts()